In [1]:
pip install transformers torch

In [2]:
pip install transformers torch faiss-cpu gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [33]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
from transformers import TrainingArguments, Trainer
import faiss
from transformers import BertModel
from datetime import datetime
import random
import gradio as gr  # Ensure gradio is imported

In [24]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)

In [25]:
# Function to get embeddings from BERT
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # Use the hidden states from the last layer
    hidden_states = outputs.hidden_states[-1]
    return outputs.last_hidden_state.mean(dim=1).numpy()

In [17]:
# Function to get a response from BERT
'''
def get_bert_response(input_text):
    inputs = tokenizer(input_text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    logits = outputs.logits
    response = torch.argmax(logits, dim=1).item()
    return response
'''

In [26]:
# Function to add context awareness
def get_greeting():
    current_hour = datetime.now().hour
    if current_hour < 12:
        return "Good morning!"
    elif 12 <= current_hour < 18:
        return "Good afternoon!"
    else:
        return "Good evening!"

In [60]:
# Simulate data from a wearable device
def get_wearable_data():
    heart_rate = random.randint(60, 100)  # Simulated heart rate
    steps = random.randint(1000, 10000)  # Simulated step count
    sleep_hours = random.uniform(4, 9)  # Simulated sleep hours
    activity_minutes = random.randint(30, 120)  # Simulated active minutes
    return heart_rate, steps, sleep_hours, activity_minutes

In [49]:
# Sample medical documents (for demonstration purposes)
documents = [
    "Regular exercise can help improve your cardiovascular health.",
    "A balanced diet is essential for maintaining good health.",
    "It's important to stay hydrated, especially during hot weather.",
    "Consult a doctor if you experience persistent symptoms."
]

In [50]:
# Create FAISS index
dimension = 768  # BERT embedding dimension
index = faiss.IndexFlatL2(dimension)
# Add document embeddings to the FAISS index
for doc in documents:
    embedding = get_embeddings(doc)
    index.add(embedding)

In [51]:
# General health advice
general_health_advice = [
    "Remember to get at least 7-8 hours of sleep each night.",
    "Stay active by engaging in regular physical exercise.",
    "Eat a variety of foods to ensure you're getting a balanced diet.",
    "Drink plenty of water to stay hydrated throughout the day."
]


In [52]:
# Function to search for the most relevant document
def search_documents(query):
    query_embedding = get_embeddings(query)
    distances, indices = index.search(query_embedding, k=1)
    return documents[indices[0][0]]

In [53]:
# Function to provide general health advice
def get_general_health_advice():
    return random.choice(general_health_advice)

In [57]:
# Function to handle user input
def chatbot_response(user_input):
    greeting = get_greeting()
    user_input_lower = user_input.lower()

    if user_input_lower in ["hi", "hello", "hey"]:
        return f"{greeting} How can I assist you with your health today?"
    elif "wearable data" in user_input_lower:
        heart_rate, steps, sleep_hours, activity_minutes = get_wearable_data()
        return (f"{greeting} Your current heart rate is {heart_rate} bpm, "
                f"you have taken {steps} steps today, "
                f"you slept for {sleep_hours:.1f} hours last night, "
                f"and you have been active for {activity_minutes} minutes today. "
                "Is there anything specific you'd like to know more about?")
    elif "advice" in user_input_lower:
        return f"{greeting} {get_general_health_advice()} Do you have any specific health concerns?"
    elif "symptom" in user_input_lower:
        return f"{greeting} For specific symptoms, it's best to consult with a healthcare professional. I can provide general advice, but a doctor will give you the best treatment plan. Can you describe your symptoms in more detail?"
    elif "heart rate" in user_input_lower or "current heart rate" in user_input_lower:
        heart_rate, _, _, _ = get_wearable_data()
        return f"{greeting} Your current heart rate is {heart_rate} bpm. How do you feel about your current heart rate?"
    elif "steps" in user_input_lower:
        _, steps, _, _ = get_wearable_data()
        return f"{greeting} You have taken {steps} steps today. Do you have a daily step goal?"
    elif "sleep" in user_input_lower:
        _, _, sleep_hours, _ = get_wearable_data()
        return f"{greeting} You slept for {sleep_hours:.1f} hours last night. How do you feel about your sleep quality?"
    elif "activity" in user_input_lower:
        _, _, _, activity_minutes = get_wearable_data()
        return f"{greeting} You have been active for {activity_minutes} minutes today. What kind of activities do you enjoy?"
    elif "normal sleep hours" in user_input_lower:
        return f"{greeting} Most adults need 7-9 hours of sleep per night to function at their best. How many hours of sleep do you usually get?"
    else:
        response = search_documents(user_input)
        return f"{greeting} {response} Is there anything else you'd like to ask?"

In [61]:
# Create Gradio interface
iface = gr.Interface(
    fn=chatbot_response,
    inputs="text",
    outputs="text",
    title="Healthcare Chatbot",
    description="An AI-based adaptive healthcare chatbot that integrates wearable data and provides personalized health advice.",
)

In [62]:
# Start the Gradio interface
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://68b2b944981ee160bf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [9]:
# Start the conversation with context-aware greeting
print(f"{get_greeting()} I am your AI-based adaptive healthcare chatbot. Type 'quit' to exit.")


Good evening! I am your AI-based adaptive healthcare chatbot. Type 'quit' to exit.


In [10]:
# Main loop to handle conversation and wearable data
while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        print("Goodbye! Take care of your health!")
        break
    elif "wearable data" in user_input.lower():
        heart_rate, steps = get_wearable_data()
        print(f"Your current heart rate is {heart_rate} bpm and you have taken {steps} steps today.")
    else:
        # Use the classifier pipeline for generating text-based responses
        result = classifier(user_input)[0]  # Get the prediction
        #response = get_bert_response(user_input)
        response = result['label']  # Extract the label (e.g., "LABEL_0", "LABEL_1")
        print(f"Chatbot: {response}")

        # Map labels to meaningful responses (customize as needed)
        if response == "LABEL_0":
            print("Chatbot: I understand.")
        elif response == "LABEL_1":
            print("Chatbot: That's interesting!")
        else:
            print("Chatbot: I'm not sure how to respond to that.")



You: hello
Chatbot: LABEL_0
Chatbot: I understand.
You: how are you
Chatbot: LABEL_0
Chatbot: I understand.


KeyboardInterrupt: Interrupted by user